This is about how things work for surrogate AL. First make sure that you have the correct env setup. On Perlmutter this is done by 
```
module load conda
conda create --prefix /pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/conda_env/rose python=3.11
conda activate /pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/conda_env/rose
pip install radical.pilot
radical-stack

# Install radical.flow
# Don't forget to install dask!!
pip install dask
pip install "dask[distributed]" --upgrade

cd /pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/radical.flow
##### Modify code in radical flow #####
pip install -e .

# also need to copy the version file:
cp /pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/radical.flow/VERSION /pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/radical.flow/src/radical/flow/

#Otherwise see the following error message:
#Traceback (most recent call last):
#  File "/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/rose_github/rose_experiment/run_me_async.py", line 5, in <module>
#    from rose.learner import ActiveLearner
#  File "/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/rose_github/rose/__init__.py", line 2, in <module>
#    from rose.learner import ActiveLearner
#  File "/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/rose_github/rose/learner.py", line 8, in <module>
#    from radical.flow import RadicalExecutionBackend, WorkflowEngine
#  File "/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/radical.flow/src/radical/flow/__init__.py", line 20, in <module>
#    = _ru.get_version(_mod_root)
#      ^^^^^^^^^^^^^^^^^^^^^^^^^^
#  File "/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/conda_env/rose/lib/python3.11/site-packages/radical/utils/get_version.py", line 72, in get_version
#    raise RuntimeError("Cannot determine version from %s (%s)"
#RuntimeError: Cannot determine version from ['/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/radical.flow/src/radical/flow'] (FileNotFoundError(2, 'No such file or directory'))

# Install rose
git clone https://github.com/radical-cybertools/ROSE.git rose_github
cd rose_github/
git checkout feature/rose_radical.flow
pip install -e .

mkdir rose_experiment; cd rose_experiment


# For nano-confinement project, add the following dependency:
pip install GPy
pip3 install torch torchvision torchaudio

# To expose this to jupyter notebook at jupyterlab on Perlmutter, do the following:
conda install ipykernel
python -m ipykernel install --user --name rose --display-name "Python ROSE conda env"
```

I didn't change the content of `model/model.py`, but did code refactoring for Fanbo's notebook and get the following five files:
```
train.py
This is the training task in terms of rose

active.py
This is the active learning task in terms of rose

bootstrap.py
This is the dataset preparing task. Run it only once (currently for each pipeline need to run it once, but actually we don't need that, so need to reorganize code a bit in the future)

run_rose.py
This is the main rose interface, about how we run parallel pipelines

utils.py
This is some helping function copy and paste from Fanbo's code
```

Currently to enable multiple pipelines running at the same time, I use a config file to control for each pipeline, the hyper-parameter for training/active learning. This is in `config/xxx.yaml`. 

Basically the model type could also be merged into this file but I have not done that.

Then if we want to run, for example, n pipelines at the same time, we just need to prepare n config files, one for each pipeline.

One important change compared with Fanbo is, we need to save and load model between task. Earlier there is a bug with gpr model saving but it is fixed now using Gpy's API. For bnn, I have not fully tested it, and there could be bug with pickle and CPU/GPU transfer.

Another important change is the data. Similarly the training dataset and the remaining dataset need to be saved after active learning task and read in training task. Of course an alternative way to do that is to use a fixed input, and only save index instead of the value. However as in the future we want to include real simulation, saving dataset will make it more forward compatible.

Below is the typical filesystem hierarchy in output dir:
```
workflow_root_dir
-pipeline_dir_1 controled by config, model
-pipeline_dir_2
-pipeline_dir_n
    -iter_001: test_data, train_data, remaining_data, model.ckpt, metric.json
	-iter_m: train_data, remaining_data, model.ckpt, metric.json
```

Notice here for each pipeline, the test data only have one copy, which is saved in iter_001 folder, and train_data, remaining_data will be saved in all iter_xxx folder (they will be different as iteration increase). 

In addition to the data, each iter_xx folder also saves the model and metric for reprod and later data processing.

Currently there is still some minor changes I need to make, for each, gpr takes a n_new_sample, but it should be merged into config.

Now you can start executing the run_rose.py. Here I just copy and paste the content here and execute it in notebook, but normally running in cmd on HPC cluster should be preferred.

In [3]:
!source ../../conda_rose.sh
#%env RADICAL_REPORT=FALSE
#%env RADICAL_LOG_LVL=DEBUG
#%env RADICAL_PROFILE=TRUE
#%env RADICAL_SMT=1
#!which python
#!python -V
#!radical-stack


  python               : /pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/conda_env/rose/bin/python3
  pythonpath           : /opt/nersc/pymon
  version              : 3.11.11
  virtualenv           : /pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/conda_env/rose

  radical.analytics    : 1.101.0
  radical.gtod         : 1.100.1
  radical.pilot        : 1.102.0
  radical.utils        : 1.100.0



In [6]:
!which python

/usr/bin/python


In [4]:
import os, sys

import radical.pilot as rp

from rose.learner import ActiveLearner
from rose.engine import Task, ResourceEngine

engine = ResourceEngine({'resource': 'nersc.perlmutter_gpu', 
                         'runtime' : 60, 
                         'access_schema':'interactive',
                         'project' : "m2616_g",
                         'cores'   : 64, 
                         'gpus'    : 4})

learner = ActiveLearner(engine=engine)
code_path = f'{sys.executable} {os.getcwd()}'

#FIXME!! I am utility task!!
@learner.simulation_task
def bootstrap(*args, pipeline_dir, input_data_dir):
    return Task(executable=f'{code_path}/bootstrap.py --pipeline_dir {pipeline_dir} --input_data_dir {input_data_dir}')
    
#FIXME!! For this two tasks, need to specify if using GPU, and other variables!!
@learner.training_task
def training(*args, model, config_file, iteration, pipeline_dir):
    return Task(executable=f'{code_path}/train.py --model {model} --config {config_file} --iteration {iteration} --pipeline_dir {pipeline_dir}')

@learner.active_learn_task
def active_learning(*args, model, config_file, iteration, pipeline_dir, num_new_samples):
    return Task(executable=f'{code_path}/active.py --model {model} --config {config_file} --iteration {iteration} --pipeline_dir {pipeline_dir} --n_new_samples {num_new_samples}')

def teach_single_pipeline(model, input_data_dir, config_file, pipeline_dir, num_iter, num_new_samples):
    iter_id = 1
    print("Start doing bootstrap (only once!)")
    bs = bootstrap(pipeline_dir=pipeline_dir, input_data_dir=input_data_dir)
    bs.result()
    while iter_id < num_iter:   # n iter means in total n traning and n-1 al
        print(f"Start doing iteration {iter_id}")
        train = training(model=model, config_file=config_file, iteration=iter_id, pipeline_dir=pipeline_dir)
        if iter_id == num_iter - 1:
            train.result()
        else:
            active_learn = active_learning(train, model=model, config_file=config_file, iteration=iter_id, pipeline_dir=pipeline_dir, num_new_samples=num_new_samples)
            active_learn.result()
        iter_id += 1

def teach():
    submitted_pipelines = []
    async_pipeline = learner.as_async(teach_single_pipeline)
    model = "gpr"
    input_data_dir = "/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/nano-confinement/surrogate_AL/data/"
    conf_list = ["/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/nano-confinement/rose_exp_1/config/gpr.yaml", "/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/nano-confinement/rose_exp_1/config/gpr_01.yaml"]
    pipeline_dir_list = ["/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/nano-confinement/rose_exp_1/experiment/test_04", "/pscratch/sd/u/usatlas/globus-compute-test/Tianle_test/nano-confinement/rose_exp_1/experiment/test_05"]
    for config_file, pipeline_dir in zip(conf_list, pipeline_dir_list):
        submitted_pipelines.append(async_pipeline(model=model, input_data_dir=input_data_dir, config_file=config_file, pipeline_dir=pipeline_dir, num_iter=10, num_new_samples=10))
    [p.result() for p in submitted_pipelines]

teach()
engine.shutdown()


Resource Engine Failed to start, terminating



FileNotFoundError: [Errno 2] No such file or directory: 'radical-pilot-bridge'